<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/KD_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Aug 24 03:36:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone -b develop-jose https://github.com/jmarrietar/Knowledge-Distillation-Zoo.git

Cloning into 'Knowledge-Distillation-Zoo'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 193 (delta 71), reused 54 (delta 15), pack-reused 53
Receiving objects: 100% (193/193), 85.34 KiB | 734.00 KiB/s, done.
Resolving deltas: 100% (103/103), done.


In [2]:
cd Knowledge-Distillation-Zoo

/content/Knowledge-Distillation-Zoo


In [4]:
#!pip install cloud-tpu-client==0.10 torch==1.8.1 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl tensorboard-plugin-profile

In [5]:
#!pip uninstall torch -y
#!pip install torch==1.8.1

In [3]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import os
import sys
import time
import logging
import argparse
import numpy as np
from itertools import chain

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import torchvision.datasets as dst
import torchvision

from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

from google.colab import auth, drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [5]:
def accuracy(output, target, topk=(1,)):
	"""Computes the precision@k for the specified values of k"""
	maxk = max(topk)
	batch_size = target.size(0)

	_, pred = output.topk(maxk, 1, True, True)
	pred    = pred.t()
	correct = pred.eq(target.view(1, -1).expand_as(pred))

	res = []
	for k in topk:
		correct_k = correct[:k].view(-1).float().sum(0)
		res.append(correct_k.mul_(100.0 / batch_size))
	return res

In [6]:
# Mount Drive
drive.mount('/content/drive')
auth.authenticate_user()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
cd ..

/content


In [8]:
import gdown

def download(data, url):
    # Download dataset
    import zipfile
    url = url
    output = "{}.zip".format(data)
    gdown.download(url, output, quiet=False)

    # Uncompress dataset
    local_zip = '{}.zip'.format(data)
    zip_ref = zipfile.ZipFile(local_zip, "r")
    zip_ref.extractall()
    zip_ref.close()

In [9]:
def get_model_property(key):
  default_model_property = {
      'img_dim': 224,
      'model_fn': getattr(torchvision.models, FLAGS.model)
  }
  model_properties = {
      'inception_v3': {
          'img_dim': 299,
          'model_fn': lambda: torchvision.models.inception_v3(aux_logits=False)
      },
  }
  model_fn = model_properties.get(FLAGS.model, default_model_property)[key]
  return model_fn

In [13]:
"""
def adjust_lr_init(optimizer, epoch):
	scale   = 0.1
	lr_list = [args.lr*scale] * 30
	lr_list += [args.lr*scale*scale] * 10
	lr_list += [args.lr*scale*scale*scale] * 10

	lr = lr_list[epoch-1]
	logging.info('Epoch: {}  lr: {:.4f}'.format(epoch, lr))
	for param_group in optimizer.param_groups:
		param_group['lr'] = lr
"""

"\ndef adjust_lr_init(optimizer, epoch):\n\tscale   = 0.1\n\tlr_list = [args.lr*scale] * 30\n\tlr_list += [args.lr*scale*scale] * 10\n\tlr_list += [args.lr*scale*scale*scale] * 10\n\n\tlr = lr_list[epoch-1]\n\tlogging.info('Epoch: {}  lr: {:.4f}'.format(epoch, lr))\n\tfor param_group in optimizer.param_groups:\n\t\tparam_group['lr'] = lr\n"

In [10]:
data_samples = {
    "sample@200": "https://drive.google.com/uc?id=1FfV7YyDJvNUCDP5r3-8iQfZ2-xJp_pgb",
    "sample@500": "https://drive.google.com/uc?id=1dHwUqpmSogEdjAB9rwDUL-OKFRUcVXte",
    "sample@1000": "https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2",
    "sample@2000": "https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F",
    "sample@3000": "https://drive.google.com/uc?id=1_yre5K9YYvJgSrT4xvrI8eD_htucIywA",
    "sample@4000_images": "https://drive.google.com/uc?id=1dqVB8EozEpwWzyuU80AauoQmsiw3Gtm2",
    "sample@20000": "https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO",
    "val-voets-tf": "https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv",
    "train_voets": "https://drive.google.com/uc?id=1AmcFh1MOOZ6aqKm2eO7XEdgmIEqHKTZ5",
    "voets_test_images": "https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd"
}

In [11]:
LABELED = 'sample@1000'

In [12]:
URL_LABELED = data_samples[LABELED]
download(LABELED, URL_LABELED)

Downloading...
From: https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2
To: /content/sample@1000.zip
108MB [00:01, 61.1MB/s]


In [13]:
!wget "https://raw.githubusercontent.com/pytorch/xla/master/test/args_parse.py"

--2021-08-24 03:37:03--  https://raw.githubusercontent.com/pytorch/xla/master/test/args_parse.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2036 (2.0K) [text/plain]
Saving to: ‘args_parse.py.1’

args_parse.py.1     100%[===================>]   1.99K  --.-KB/s    in 0s      

2021-08-24 03:37:04 (43.2 MB/s) - ‘args_parse.py.1’ saved [2036/2036]



In [14]:
import args_parse

SUPPORTED_MODELS = [
    'alexnet', 'densenet121', 'densenet161', 'densenet169', 'densenet201',
    'inception_v3', 'resnet101', 'resnet152', 'resnet18', 'resnet34',
    'resnet50', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13',
    'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn'
]

MODEL_OPTS = {
    '--model': {
        'choices': SUPPORTED_MODELS,
        'default': 'resnet50',
    },
    '--test_set_batch_size': {
        'type': int,
    },
    '--lr_scheduler_type': {
        'type': str,
    },
    '--lr_scheduler_divide_every_n_epochs': {
        'type': int,
    },
    '--lr_scheduler_divisor': {
        'type': int,
    },
    '--test_only_at_end': {
        'action': 'store_true',
    },
}

In [15]:
FLAGS = args_parse.parse_common_options(
    datadir=LABELED,
    batch_size=None,
    num_epochs=None,
    momentum=None,
    lr=None,
    target_accuracy=None,
    profiler_port=9012,
    opts=MODEL_OPTS.items(),
)

In [16]:
EPOCHS = 200

In [17]:
FLAGS.fake_data = False
FLAGS.num_epochs = EPOCHS
FLAGS.batch_size = 64
FLAGS.log_steps = 100
FLAGS.num_cores = 8

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [55]:
#model1 = get_model_property('model_fn')().to(device)
model1 = torchvision.models.mobilenet_v2(pretrained=True).to(device)

model1.classifier = nn.Sequential(
      nn.Linear(1280, 512),
      nn.Linear(512, 1),
      nn.Sigmoid()
  ).to(device)

In [21]:
state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/SimCLR/models/SimCLR-1-DR-pytorch/net-DR-SimCLR-Finetuned-Test-{}.pt'.format(LABELED))

In [22]:
state_dict.keys()

dict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.wei

In [71]:
#log = model1.load_state_dict(state_dict, strict=False)
#print(log)

In [23]:
model2 = get_model_property('model_fn')().to(device)

model2.fc = nn.Sequential(
      nn.Linear(2048, 512),
      nn.Linear(512, 1),
      nn.Sigmoid()
  ).to(device)

In [24]:
log = model2.load_state_dict(state_dict, strict=False)
print(log)

<All keys matched successfully>


In [25]:
parser = argparse.ArgumentParser(description='train kd')

# various path
parser.add_argument('--save_root', type=str, default='./results', help='models and logs are saved here')
parser.add_argument('--img_root', type=str, default='./datasets', help='path name of image dataset')
parser.add_argument('--s_init', type=str, required=True, help='initial parameters of student model')
parser.add_argument('--t_model', type=str, required=True, help='path name of teacher model')

# training hyper parameters
parser.add_argument('--print_freq', type=int, default=50, help='frequency of showing training results on console')
parser.add_argument('--epochs', type=int, default=200, help='number of total epochs to run')
parser.add_argument('--batch_size', type=int, default=128, help='The size of batch')
parser.add_argument('--lr', type=float, default=0.1, help='initial learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--weight_decay', type=float, default=1e-4, help='weight decay')
parser.add_argument('--num_class', type=int, default=10, help='number of classes')
parser.add_argument('--cuda', type=int, default=1)

# others
parser.add_argument('--seed', type=int, default=2, help='random seed')
parser.add_argument('--note', type=str, default='try', help='note for this run')

# net and dataset choosen
parser.add_argument('--data_name', type=str, required=True, help='name of dataset') # cifar10/cifar100
parser.add_argument('--t_name', type=str, required=True, help='name of teacher')    # resnet20/resnet110
parser.add_argument('--s_name', type=str, required=True, help='name of student')    # resnet20/resnet110

# hyperparameter
parser.add_argument('--kd_mode', type=str, required=True, help='mode of kd, which can be:'
															   'logits/st/at/fitnet/nst/pkt/fsp/rkd/ab/'
															   'sp/sobolev/cc/lwm/irg/vid/ofd/afd')
parser.add_argument('--lambda_kd', type=float, default=1.0, help='trade-off parameter for kd loss')
parser.add_argument('--T', type=float, default=4.0, help='temperature for ST')
parser.add_argument('--p', type=float, default=2.0, help='power for AT')
parser.add_argument('--w_dist', type=float, default=25.0, help='weight for RKD distance')
parser.add_argument('--w_angle', type=float, default=50.0, help='weight for RKD angle')
parser.add_argument('--m', type=float, default=2.0, help='margin for AB')
parser.add_argument('--gamma', type=float, default=0.4, help='gamma in Gaussian RBF for CC')
parser.add_argument('--P_order', type=int, default=2, help='P-order Taylor series of Gaussian RBF for CC')
parser.add_argument('--w_irg_vert', type=float, default=0.1, help='weight for IRG vertex')
parser.add_argument('--w_irg_edge', type=float, default=5.0, help='weight for IRG edge')
parser.add_argument('--w_irg_tran', type=float, default=5.0, help='weight for IRG transformation')
parser.add_argument('--sf', type=float, default=1.0, help='scale factor for VID, i.e. mid_channels = sf * out_channels')
parser.add_argument('--init_var', type=float, default=5.0, help='initial variance for VID')
parser.add_argument('--att_f', type=float, default=1.0, help='attention factor of mid_channels for AFD')


_StoreAction(option_strings=['--att_f'], dest='att_f', nargs=None, const=None, default=1.0, type=<class 'float'>, choices=None, help='attention factor of mid_channels for AFD', metavar=None)

In [92]:
args, unparsed = parser.parse_known_args(['--save_root', './results/logits/',
                                          '--t_model', './results/base/base-c10-r110/model_best.pth.tar',
                                          '--s_init', './results/base/base-c10-r20/initial_r20.pth.tar',
                                          '--data_name', 'DR',
                                          '--batch_size', '32',
                                          '--t_name', 'resnet110', 
                                          '--s_name', 'resnet20',
                                          '--num_class', '2',
                                          '--kd_mode', 'logits', 
                                          '--lambda_kd', '0.1',
                                          '--note', 'logits-c10-r110-r20',
                                          '--print_freq', '30',
                                          '--lr', '0.0001', 
                                          ])

In [27]:
class Logits(nn.Module):
	'''
	Do Deep Nets Really Need to be Deep?
	http://papers.nips.cc/paper/5484-do-deep-nets-really-need-to-be-deep.pdf
	'''
	def __init__(self):
		super(Logits, self).__init__()

	def forward(self, out_s, out_t):
		loss = F.mse_loss(out_s, out_t)

		return loss

In [28]:
"""
def adjust_lr(optimizer, epoch):
	scale   = 0.1
	lr_list =  [args.lr] * 100
	lr_list += [args.lr*scale] * 50
	lr_list += [args.lr*scale*scale] * 50

	lr = lr_list[epoch-1]
	logging.info('Epoch: {}  lr: {:.3f}'.format(epoch, lr))
	for param_group in optimizer.param_groups:
		param_group['lr'] = lr
"""

"\ndef adjust_lr(optimizer, epoch):\n\tscale   = 0.1\n\tlr_list =  [args.lr] * 100\n\tlr_list += [args.lr*scale] * 50\n\tlr_list += [args.lr*scale*scale] * 50\n\n\tlr = lr_list[epoch-1]\n\tlogging.info('Epoch: {}  lr: {:.3f}'.format(epoch, lr))\n\tfor param_group in optimizer.param_groups:\n\t\tparam_group['lr'] = lr\n"

In [56]:
snet = model1

In [57]:
tnet = model2

In [58]:
for param in tnet.parameters():
  param.requires_grad = False

In [59]:
logging.info('Teacher: %s', tnet)
logging.info('-----------------------------------------------')

In [60]:
criterionKD = Logits()

In [61]:
criterionCls = torch.nn.BCELoss()

In [62]:
optimizer = torch.optim.SGD(snet.parameters(),
              lr = args.lr, 
              momentum = args.momentum, 
              weight_decay = args.weight_decay,
              nesterov = True)

In [63]:
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

In [64]:
train_transform = transforms.Compose([
    transforms.Pad(4, padding_mode='reflect'),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,std=std)
  ])

In [65]:
img_dim = get_model_property('img_dim')

In [66]:
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [67]:
train_dataset = torchvision.datasets.ImageFolder(
    os.path.join(FLAGS.datadir, 'train'),
    transforms.Compose([
        #transforms.RandomResizedCrop(img_dim),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

In [68]:
# Define data loader
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size, 
    shuffle=True, 
    num_workers=2, 
    pin_memory=True)

In [69]:
# warp nets and criterions for train and test
nets = {'snet':snet, 'tnet':tnet}
criterions = {'criterionCls':criterionCls, 'criterionKD':criterionKD}

In [70]:
cd Knowledge-Distillation-Zoo

/content/Knowledge-Distillation-Zoo


In [71]:
from utils import AverageMeter, accuracy, transform_time
from utils import load_pretrained_model, save_checkpoint
from utils import create_exp_dir, count_parameters_in_MB

In [72]:
cd ..

/content


In [93]:
def train(train_loader, nets, optimizer, criterions, epoch):
    batch_time = AverageMeter()
    data_time  = AverageMeter()
    cls_losses = AverageMeter()
    kd_losses  = AverageMeter()
    top1       = AverageMeter()
    top5       = AverageMeter()

    snet = nets['snet']
    tnet = nets['tnet']

    criterionCls = criterions['criterionCls']
    criterionKD  = criterions['criterionKD']


    snet.train()

    if args.kd_mode in ['vid', 'ofd']:
        for i in range(1,4):
            criterionKD[i].train()

    end = time.time()

    for i, (img, target) in enumerate(train_loader, start=1):
        data_time.update(time.time() - end)

        if args.cuda:
            img = img.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)

        if args.kd_mode in ['sobolev', 'lwm']:
            img.requires_grad = True

        out_s = snet(img)
        out_t = tnet(img)

        target = target.unsqueeze(1)
        target = target.to(torch.float32)

        cls_loss = criterionCls(out_s, target)

        if args.kd_mode in ['logits', 'st']:
            #kd_loss = criterionKD(out_s, out_t.detach()) * args.lambda_kd
            kd_loss = criterionCls(out_s, out_t.detach()) # YOOOO

        loss = cls_loss + kd_loss

        prec1 = accuracy(out_s, target, topk=(1,))

        cls_losses.update(cls_loss.item(), img.size(0))
        kd_losses.update(kd_loss.item(), img.size(0))
        top1.update(prec1[0].item(), img.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            log_str = ('Epoch[{0}]:[{1:03}/{2:03}] '
                        'Time:{batch_time.val:.4f} '
                        'Data:{data_time.val:.4f}  '
                        'Cls:{cls_losses.val:.4f}({cls_losses.avg:.4f})  '
                        'KD:{kd_losses.val:.4f}({kd_losses.avg:.4f})  '
                        'prec@1:{top1.val:.2f}({top1.avg:.2f})  '.format(
                        epoch, i, len(train_loader), batch_time=batch_time, data_time=data_time,
                        cls_losses=cls_losses, kd_losses=kd_losses, top1=top1))
            logging.info(log_str)
            print(log_str)

In [74]:
"""
Idea: 
  - Comparar realizando Knowledge distillation solo de LABELED 1000 
  VS Knowledge distilattion de la UNLABELED 5000 (Solo el loss de consistency) + 
  despues un Finetuning de la Unlabeled
"""

'\nIdea: \n  - Comparar realizando Knowledge distillation solo de LABELED 1000 \n  VS Knowledge distilattion de la UNLABELED 5000 (Solo el loss de consistency) + \n  despues un Finetuning de la Unlabeled\n'

In [90]:
pwd

'/content'

In [ ]:
best_top1 = 0
for epoch in range(1, args.epochs+1):
  #adjust_lr(optimizer, epoch)

  # train one epoch
  epoch_start_time = time.time()
  train(train_loader, nets, optimizer, criterions, epoch)

  epoch_duration = time.time() - epoch_start_time
  logging.info('Epoch time: {}s'.format(int(epoch_duration)))
  print('Epoch time: {}s'.format(int(epoch_duration)))

  """
  # save model
  is_best = False
  if test_top1 > best_top1:
    best_top1 = test_top1
    is_best = True
  logging.info('Saving models......')
  save_checkpoint({
    'epoch': epoch,
    'snet': snet.state_dict(),
    'tnet': tnet.state_dict(),
    'prec@1': test_top1,
  }, is_best, args.save_root)
  """

Epoch[1]:[030/032] Time:0.2159 Data:0.0001  Cls:0.5394(0.6114)  KD:0.5834(0.6052)  prec@1:53.12(50.31)  
Epoch time: 7s
Epoch[2]:[030/032] Time:0.2146 Data:0.0001  Cls:0.6247(0.6151)  KD:0.5972(0.6091)  prec@1:40.62(49.58)  
Epoch time: 7s
Epoch[3]:[030/032] Time:0.2147 Data:0.0001  Cls:0.6102(0.6200)  KD:0.5707(0.6103)  prec@1:46.88(49.69)  
Epoch time: 7s
Epoch[4]:[030/032] Time:0.2143 Data:0.0001  Cls:0.6476(0.6158)  KD:0.6531(0.6089)  prec@1:65.62(50.10)  
Epoch time: 7s
Epoch[5]:[030/032] Time:0.2169 Data:0.0001  Cls:0.6379(0.6123)  KD:0.5752(0.6046)  prec@1:40.62(49.90)  
Epoch time: 7s
Epoch[6]:[030/032] Time:0.2119 Data:0.0001  Cls:0.6875(0.6109)  KD:0.6589(0.6052)  prec@1:56.25(50.21)  
Epoch time: 7s
Epoch[7]:[030/032] Time:0.2163 Data:0.0001  Cls:0.5803(0.6135)  KD:0.5853(0.6045)  prec@1:40.62(50.00)  
Epoch time: 7s
Epoch[8]:[030/032] Time:0.2154 Data:0.0001  Cls:0.6193(0.6102)  KD:0.6452(0.6040)  prec@1:50.00(49.90)  
Epoch time: 7s


# Test

In [ ]:
model = snet

In [ ]:
import zipfile
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
test_data = {
    "voets_test_images": "https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd",
    "messidor2": "https://drive.google.com/uc?id=1HaUAxDtN4BNj0hpH8QYGmiX39Va-Ke8p",
}

TEST_DATASET = 'messidor2'
URL_TEST_DATASET = test_data[TEST_DATASET]

In [ ]:
!gdown $URL_TEST_DATASET

In [ ]:
local_zip = '{}.zip'.format(TEST_DATASET)
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2471, 0.2435, 0.2616)

In [ ]:
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar10_mean, std=cifar10_std)  # What happens if I change This 
])

In [ ]:
test_dataset = datasets.ImageFolder(root="messidor2", transform=transform_val) # root: test/messidor2

In [ ]:
test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    num_workers=1,
    shuffle=False)

loader = test_loader

In [ ]:
y_true = []
y_pred = []

with torch.no_grad():
    with tqdm(total=len(loader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(loader):

            model.eval()

            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)

            output = output.to(device)

            # Yo le agrege esto
            y_true.append(targets.cpu().detach().numpy()[0])
            y_pred.append(output.cpu().detach().numpy()[0][0])

            pbar.update(1)

In [ ]:
import numpy as np
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
metrics.auc(fpr, tpr)

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mlxtend.plotting import plot_confusion_matrix 
# %matplotlib inline

#cm=metrics.confusion_matrix(y_true, y_pred)
auc = metrics.auc(fpr, tpr)
#print('AUC: %.3f' % auc)
#print('Accuracy: {}'.format(accuracy_score(y_true, y_pred)))

# Plot ROC curve
lw = 2
sns.set_style({'axes.grid' : False})
sns.set_style("darkgrid")
ax1 = sns.lineplot(fpr, tpr, color='darkorange',
        lw=lw, label='AUC = %0.2f' % auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
ax1.set_title('Receiver operating characteristic')
ax1.set(xlabel='False Positive Rate', ylabel='True Positive Rate')
plt.legend(loc="lower right")
plt.show()